In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Ucitavanje i sredivanje dataseta

### Ucitavanje svih excel fajlova u jedan dataframe

In [ ]:
new_df = pd.DataFrame()

In [ ]:
for i in range(1998,2022):
    df = pd.read_csv('input/atp_matches_' + str(i) + '.csv')
    df['year'] = i
    new_df = pd.concat([new_df,df], axis=0)
new_df.shape

In [ ]:
for i in range(2010,2022):
    df = pd.read_csv('input/atp_matches_qual_chall_' + str(i) + '.csv')
    df['year'] = i
    new_df = pd.concat([new_df,df], axis=0)
new_df.shape

### Provera nedostajucih vrednosti

In [ ]:
new_df.columns

In [ ]:
new_df = new_df[~(new_df['tourney_name'].str.contains('Davis'))]

In [ ]:
new_df.isna().sum()

In [ ]:
surface_list = new_df.surface.unique() #provera podataka o podlozi i brisanje redova sa nedostajucim vrednostima
print(surface_list)
new_df = new_df.dropna(subset=['surface'])
surface_list = new_df.surface.unique()
print(surface_list)

In [ ]:
new_df.draw_size.unique()

In [ ]:
new_df.loc[new_df['winner_seed'].isna(),'winner_seed'] = 40 #postavljanje reda broja nosioca za one koji nedotaju na 40
new_df.loc[new_df['loser_seed'].isna(), 'loser_seed'] = 40

In [ ]:
print(new_df['winner_entry'].unique())
new_df.loc[new_df['winner_entry'].isna(), 'winner_entry'] = 'A'
new_df.loc[new_df['loser_entry'].isna(), 'loser_entry'] = 'A'

In [ ]:
print(new_df[new_df['winner_hand'].isna()].shape[0])
new_df.loc[new_df['winner_hand'].isna(), 'winner_hand'] = 'U'
new_df.loc[new_df['loser_hand'].isna(), 'loser_hand'] = 'U'
new_df['winner_hand'].unique()

In [ ]:
#postavljanje visine gde nedostaje na na prosecnu visinu pobednika, odnosno gubitnika
w_ht = new_df['winner_ht'].mean()
print('Prosecna visina pobednika: ' + str(w_ht))
new_df.loc[new_df['winner_ht'].isna(), 'winner_ht'] = w_ht
l_ht = new_df['loser_ht'].mean()
print('Prosecna visina gubitnika: ' + str(l_ht))
new_df.loc[new_df['loser_ht'].isna(), 'loser_ht'] = l_ht

In [ ]:
#postavljanje starosti igraca gde nedostaje na na prosecnu starost pobednika, odnosno gubitnika
w_age = new_df['winner_age'].mean() 
print('Prosecna starost pobdenika: ' + str(w_age))
new_df.loc[new_df['winner_age'].isna(), 'winner_age'] = w_age
l_age = new_df['loser_age'].mean()
print('Prosecna starost gubitnika' + str(l_age))
new_df.loc[new_df['loser_age'].isna(), 'loser_age'] = l_age

In [ ]:
#postavljanje ranka na 2000, sto veci rank igrac je losiji
new_df.loc[new_df['winner_rank'].isna(), 'winner_rank'] = 2000
new_df.loc[new_df['loser_rank'].isna(), 'loser_rank'] = 2000

In [ ]:
# postavljanje broja bodova na 0
new_df.loc[new_df['winner_rank_points'].isna(), 'winner_rank_points'] = 0
new_df.loc[new_df['loser_rank_points'].isna(), 'loser_rank_points'] = 0

In [ ]:
# brisanje meceva koji nemaju rezultat
new_df.score.unique()
new_df = new_df.dropna(subset=['score'])
new_df = new_df.loc[new_df['score'] != 'W/O']

In [ ]:
# runda turnira nema nedostajucu vrednost tako da ostaje netaknuta
new_df['round'].unique()

In [ ]:
# brisemo meceve koji nemaju duzinu trajanja
new_df = new_df.loc[~(new_df['minutes'].isna())]

In [ ]:
new_df.isna().sum()

In [ ]:
# brisemo sve redove gde su ne popunjene numericke vrednosti, broj aseva, duplih gresaka..
new_df = new_df.dropna()

In [ ]:
new_df.isna().sum()

In [ ]:
new_df.to_csv('new_data/tenis.csv', index=False)

### Analiza podataka

In [ ]:
#provera koliko pobeda ima Novak Djokovic
djokovic_id = new_df[new_df['winner_name'] == 'Novak Djokovic']['winner_id'].unique()[0]  
print('Broj pobeda Novaka Djokovica je ' + str(new_df[new_df['winner_id']==djokovic_id].shape[0]))

In [ ]:
print('Broj razlicitih pobednika je ' + str(new_df['winner_id'].nunique())) #broj igraca koji su odneli pobedu

In [ ]:
# Poredjenje odnosa ranka i procenat koliko je igraca pobedilo bolje rangiranog
print('Broj pobednika sa vecim rangom je  ' + str(new_df[new_df['loser_rank'] < new_df['winner_rank']].shape[0]))
print('Broj pobednika sa losijim rangom je ' + str(new_df[new_df['loser_rank'] > new_df['winner_rank']].shape[0]))
print('Procenat pobednika sa losijim rangom je ' + str(new_df[new_df['loser_rank'] < new_df['winner_rank']].shape[0] * 100 / new_df.shape[0]))

In [ ]:
y = new_df[['winner_age', 'winner_id']].astype(int).groupby('winner_age').count().plot(kind='bar')